In [ ]:
# FRIDAY 1: TOKENIZATION & EMBEDDINGS 
# components 
# 1. Tokenizer Convertes text <-> Numbers
# 2. Token Embedding - Numbers -> Dense vectors 
# 3. Positional embeddings - Add position information 
# 4. Testing & Visualization 


In [2]:
# imports 
import torch 
import torch.nn as nn 
import pickle
import os 
from pathlib import Path

In [4]:
# sample data 
with open('data/sample.txt',encoding='utf-8') as f:
    text = f.read()

FileNotFoundError: [Errno 2] No such file or directory: 'data/sample.txt'

In [ ]:
# component 1 : TOKENIZER 
class Tokenizer:
    """
    Character -Level tokenizer for Nepali text 
    what it does 
    - Builds a vocabulart for all uqnieu characters in the text
    - Convert text to numbers(encoding) and vice versa 
    """
    def __init__(self,text):
        # process : extract->sort->create bidirectional mapping
        self.chars = list(set(text))